In [1]:
import re
import numpy as np
import io
import pandas as pd
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from PIL import Image
import PIL.ImageOps
from wordcloud import ImageColorGenerator
import string
import unicodedata
from nltk.corpus import stopwords
import nltk
from contractions import CONTRACTION_MAP
import spacy
import random
from spacy.lang.en import English
nlp = English()

In [2]:
nlp = English()

In [3]:
import sqlite3 as sl

con = sl.connect('tweets')

apple_positive = pd.read_sql("""
                    SELECT tweet_text
                    FROM tweets
                    WHERE apple_product = '1'
                        AND pos_rev = '1'
                    """,con)
print(apple_positive.head())
print(apple_positive.shape)

                                          tweet_text
0  @jessedee Know about @fludapp ? Awesome iPad/i...
1  @swonderlin Can not wait for #iPad 2 also. The...
2  Beautifully smart and simple idea RT @madebyma...
3  Counting down the days to #sxsw plus strong Ca...
4  Great #sxsw ipad app from @madebymany: http://...
(1949, 1)
